In [1]:
import numpy           as np
import pandas          as pd
from scipy.stats    import norm
from scipy.optimize import minimize, OptimizeResult

### 1. import data

In [2]:
SOURCE = "../DataSets/cowing.xlsx"

df = pd.read_excel(SOURCE)
y = df["y"]
[x1, x2, x3] = [df["X1"], df["X2"], df["X3"]]
[p1, p2, p3] = [np.log(df["P1"]), np.log(df["P2"]), np.log(df["P3"])]

### 2. functions implementations

#### AppLogDen

In [3]:
def AppLogDen_ALS77(Pars: np.array):
    
    global y, x1, x2, x3, p1, p2, p3
    
    [alpha, beta1, beta2, beta3, sigma2u, sigma2v] = Pars[0:6]
    
    Lambda = np.sqrt( sigma2u / sigma2v )
    sigma2 = sigma2u + sigma2v
    sigma  = np.sqrt( sigma2 )
    
    eps = y - alpha - x1*beta1 - x2*beta2 - x3*beta3
    B2  = p2 - p1 + np.log(beta1) - np.log(beta2)
    B3  = p3 - p1 + np.log(beta1) - np.log(beta3)
    w1  = x1 - x2 - B2
    w2  = x1 - x3 - B3
    w   = pd.concat([w1, w2], axis=1)
    
    # in norm.pdf and norm.cdf loc=0 and scale=1 by default 
    Den = (2/sigma) * norm.pdf(eps / sigma) * norm.cdf(-Lambda * eps / sigma)
    
    return np.log(Den)

#### AppLoglikelihood

In [4]:
def AppLoglikelihood_ALS77(coefs: np.array):
    
    #~~~~ transform parametrs back true range ~~~~#
    coefs[4:6] = np.exp(coefs[4:6])
    
    #~~~~ obtain the log likelihood ~~~~#
    logDen = AppLogDen_ALS77(coefs)
    return -sum(logDen)

#### AppEstimate

In [5]:
it = 0
def _callback(vec):
    global it
    print(f"{it}\t {AppLoglikelihood_ALS77(np.copy(vec))}")    
    it += 1

In [11]:
def AppEstimate_ALS77():
    
    global y, x1, x2, x3, p1, p2, p3, it
    
    alpha    = -11 
    beta1    = 0.03
    beta2    = 1.1
    beta3    = -0.01
    
    sigma2u  = 0.01
    sigma2v  = 0.0003
    lsigma2u = np.log(sigma2u)
    lsigma2v = np.log(sigma2v)
    
    theta0 = [alpha, beta1, beta2, beta3, lsigma2u, lsigma2v]
    
    print("Iteration\t f(x)")
    rez = minimize(AppLoglikelihood_ALS77, np.array(theta0),
                   method="Powell",
                   options={"disp": True,
                            "xtol": 1e-10,
                            "ftol": 1e-10,
                            "maxiter": 20000},
                   callback=_callback)
    it = 0
    theta  = rez.x
    logMLE = rez.fun
    
    #~~~~ standard errors ~~~~#
    theta[4:6] = np.exp(theta[4:6])
    delta = 1e-6
    grad  = pd.DataFrame( np.zeros((len(y), len(theta))) )
    
    for i in range(len(theta)):
        theta1 = np.copy(theta)
        theta1[i] += delta
        grad.iloc[:, i] = (AppLogDen_ALS77(theta1) - 
                           AppLogDen_ALS77(theta )) / delta
        
    OPG  = grad.transpose().dot(grad)
    D    = np.diag( np.concatenate(([1, 1, 1, 1], theta[4:6])) )
    ster = np.sqrt( np.diag(np.linalg.inv(OPG)) )
    
    return [theta, ster, logMLE]

### 3. output

In [12]:
[theta, ster, logMLE] = AppEstimate_ALS77()
print("ans = \n")
for i in range(len(theta)):
    print("\t%.4f" % theta[i])

Iteration	 f(x)


/home/fomius2000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]
/home/fomius2000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log
  if sys.path[0] == '':


0	 47.14567748766984


/home/fomius2000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log


1	 -35.81819746718527
2	 -100.14104105349236
3	 -118.38348859675008
4	 -121.85594471590093
5	 -122.05374246084331
6	 -122.1424037300356
7	 -122.14403327985067
8	 -122.34570495344094
9	 -122.41179958033024
10	 -122.53464190912723
11	 -122.53586135681704
12	 -122.73875595200431
13	 -122.90292096521199
14	 -123.00973669437326
15	 -123.0106613968861
16	 -123.01068196581863
17	 -123.01116882631786
18	 -123.01262932100515
19	 -123.0240722772228
20	 -123.0508563423245
21	 -123.05762669980817
22	 -123.06065321185777
23	 -123.06065425925654
24	 -123.06065448176794
Optimization terminated successfully.
         Current function value: -123.060654
         Iterations: 25
         Function evaluations: 4099
ans = 

	-11.0178
	0.0402
	1.0860
	-0.0191
	0.0107
	0.0027
